# LEARNING PYTORCH WITH EXAMPLES

- 이 코드는 아래 원본 링크에서 PyTorch: Tensors를 번역 및 주석을 추가한 글입니다.
- 다른 부분은 [여기](https://github.com/pbj0812/deep_learning/tree/master/pytorch_tutorial) 에서 확인할 수 있습니다.

- 원본 링크 : [링크](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)
- 저자 : [Justin Johnson](https://github.com/jcjohnson/pytorch-examples)
- 번역 : [박범진](http://github.com/pbj0812)

## PyTorch: Tensors

Numpy는 매우 휼륭한 프레임워크이지만, 수치 연산에 있어 GPU를 활용하지 못한다. 현재의 딥러닝 네트워크에서, GPU는 종종 50배 이상의 속력을 내기도 한다, 그래서 불행하게도 numpy는 딥러닝에 적합하지 못하다.
r
이번에 우리는 가장 기초적인 PyTorch의 컨셉인 Tensor를 소개한다. PyTorch Tensor는 개념적으로 numpy 행렬과 동일하다: Tensor는 n-차원의 행렬이며, PyTorch는 Tensor를 위한 여러 함수들을 제공한다. Tensor는 계산 그래프와 gradients를 추적하지만, 그것들은 과학적인 계산에 있어 유용한 툴이다.

numpy와는 다르게, PyTorch Tensor는 수치 연산을 위해서 GPU를 사용할 수 있다. GPU에서 PyTorch Tensor를 실행하기 위해서는, 단지 새로운 datatype으로 바꿔주면 된다.

이번에 우리는 PyTorch Tensor를 사용하여 이층 구조의 랜덤데이터로 이루어진 네트워크를 구현할 것이다. numpy로 구현했던 것과 같은 방식으로 우리는 forward와 backward에 대한 구현이 필요하다.

In [1]:
# -*- coding: utf-8 -*-


import torch

In [2]:
dtype = torch.float
# device = torch.device("cpu") # GPU를 실행시킬 수 없으면 주석을 풀어준다.
device = torch.device("cuda:0") # GPU를 실행시킬 수 있으면 주석을 풀어준다.

In [3]:
# N 은 batch size; D_in 은 입력값의 차원;
# H 는 hidden 차원; D_out 은 출력값의 차원.
N, D_in, H, D_out = 64, 1000, 100, 10

In [4]:
# Creat random input and output data
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

In [5]:
# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6

In [6]:
for t in range(500):
    # Forward pass : y예측
    h = x.mm(w1) # x 행렬 곱하기 w1 행렬
    h_relu = h.clamp(min=0) # ReLU 구현
    y_pred = h_relu.mm(w2) # h_relu X w2
    
    # loss 계산
    loss = (y_pred - y).pow(2).sum().item() # (y_pred - y)^2 의 모든 값들을 더한 뒤 뽑기
    print(t, loss)
    
    # w1과 w2 간의 loss를 통한 gradients 계산을 위한 backprop
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred) # w2 gradient 계산
    grad_h_relu = grad_y_pred.mm(w2.t()) # h_relu gradient 계산
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0 # ReLU 구현
    grad_w1 = x.t().mm(grad_h) # w1 gradient 계산
    
    # gradient descent를 이용한 weights 갱신
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 26540710.0
1 20107668.0
2 15909755.0
3 12427995.0
4 9300372.0
5 6734272.0
6 4755227.5
7 3352814.0
8 2386717.5
9 1739063.875
10 1301247.75
11 1002214.625
12 792285.6875
13 640872.0
14 528206.75
15 442057.8125
16 374485.625
17 320445.5
18 276481.6875
19 240146.765625
20 209717.59375
21 184046.671875
22 162178.015625
23 143415.796875
24 127217.765625
25 113176.859375
26 100956.421875
27 90262.8984375
28 80870.7265625
29 72590.703125
30 65279.80859375
31 58804.515625
32 53057.3203125
33 47946.41015625
34 43392.24609375
35 39321.67578125
36 35680.46875
37 32415.416015625
38 29485.498046875
39 26849.388671875
40 24475.48046875
41 22336.34375
42 20404.505859375
43 18658.396484375
44 17077.9453125
45 15644.982421875
46 14345.1015625
47 13164.66015625
48 12091.0654296875
49 11113.892578125
50 10223.158203125
51 9410.9013671875
52 8669.5439453125
53 7992.2314453125
54 7372.77685546875
55 6805.92333984375
56 6286.86572265625
57 5810.66748046875
58 5373.716796875
59 4972.7099609375
60 4604.27441

431 0.00010853513231268153
432 0.00010672520147636533
433 0.00010473368456587195
434 0.00010281280265189707
435 0.00010077508341055363
436 9.880810102913529e-05
437 9.71889603533782e-05
438 9.54759307205677e-05
439 9.386254532728344e-05
440 9.236847108695656e-05
441 9.082581527763978e-05
442 8.919182437239215e-05
443 8.79083527252078e-05
444 8.610280201537535e-05
445 8.463699487037957e-05
446 8.308111864607781e-05
447 8.172496745828539e-05
448 8.058096136664972e-05
449 7.962069503264502e-05
450 7.802994514349848e-05
451 7.637708040419966e-05
452 7.524913235101849e-05
453 7.413428829750046e-05
454 7.304370228666812e-05
455 7.20074531272985e-05
456 7.081389776431024e-05
457 6.992441922193393e-05
458 6.869067874504253e-05
459 6.764510908396915e-05
460 6.675609620288014e-05
461 6.581891648238525e-05
462 6.49092035018839e-05
463 6.364137516357005e-05
464 6.280504749156535e-05
465 6.201749056344852e-05
466 6.109843525337055e-05
467 6.025991024216637e-05
468 5.9520447393879294e-05
469 5.84075